In [1]:
from gurobipy import *
import numpy as np

In [2]:
# d={(1, 5): 0,(5, 1): 0,
#  (1, 4): 39,(4, 1): 39,
#  (1, 2): 137,(2, 1): 137,
#  (1, 3): 125,(3, 1): 125,
#  (5, 4): 39,(4, 5): 39,
#  (5, 2): 137,(2, 5): 137,
#  (5, 3): 125,(3, 5): 125,
#  (2, 4): 133,(4, 2): 133,
#  (2, 3): 129,(3, 2): 129,
#  (3, 4): 229,(4, 3): 229,
#   (1, 1): 0,(5, 5): 0,
#   (2, 2): 0,(3, 3): 0,
#   (4, 4): 0}
# v = 40
# a = [0]*5
# b = [12]*5

In [3]:
# h = np.zeros([6,6,5])
# x=h
# w2 = np.zeros([6,4])
# w=w2
flag=False

All the work events have the same occurance time and the same time window from 0 to 12.

In [4]:
n=11
nc=7
large=1000
v=20
a=[0]*n
b=[12]*n

When I set random occurance time but have the same time window, the problem is infeasible.

In [28]:
# b = [12,
#  16.001918040736232,
#  17.335593256824332,
#  18.518709936731096,
#  19.669635532994683,
#  20.12356946327924,
#  20.652968866328045,
#  21.643940827275898,
#  21.72356946327932,
#  22.0849585187867,
#  24]
# tim=12 # time window is still 12 for all work events
# a=[i-tim for i in b]
# a

In [6]:
f = open('vrp10.txt')
d ={}
count = 0
for i in f.readlines():
    if count == 0:
        count += 1
        continue
    
    cur = i.split()
    for j in range(1,n):
        d[count,j]=float(cur[j])
    d[count,n]=d[count,1]
    count+=1
for i in range(1,n+1):
    d[count,i]=d[1,i]

In [7]:
# x[1,4,1] = 1
# x[4,2,1] = 1
# x[2,3,1] = 1
# x[3,5,1] = 1

# w[1,1] = 0
# w[4,1] = 0.975
# w[2,1] = 4.3
# w[3,1] = 7.525
# w[5,1] = 10.65
# obj = quicksum(quicksum(x[1,j,c]for c in range(1,4)) for j in range(1,6))
# print(obj)

In [8]:
myModel = Model("vrp")
myModel.update()

Academic license - for non-commercial use only


## Adding variables

In [9]:
x = myModel.addVars(range(1,n+1),range(1,n+1),range(1,nc+1),vtype=GRB.BINARY,lb=0.0,name='x')
w = myModel.addVars(range(1,n+1),range(1,nc+1),vtype=GRB.CONTINUOUS,lb=0.0,name='w')

## Adding constraints

$$
w_{0c}\geq F,\ w_{0\prime c}\le L,\ where\ F=0\ and\ L=12,\forall c
$$

In [10]:
for c in range(1,nc+1):
    myModel.addConstr(w[1,c]>=0)
    myModel.addConstr(w[n,c]<=12)
    if flag:
        print(c,'   ',w2[1,c]>=0,w2[n,c]<=12)

$$
\sum_{j} X_{0jc}\le1,\forall c
$$

In [11]:
for c in range(1,nc+1):
    con1 = quicksum(x[1,j,c] for j in range(1,n+1))
    myModel.addConstr(con1 <= 1)
    if flag:
        print(quicksum(h[1,j,c] for j in range(1,n+1)))

$$
\sum_{j} X_{j0\prime c}\le1,\ \forall c
$$

In [12]:
for c in range(1,nc+1):
    con2 = quicksum(x[j,n,c] for j in range(1,n+1))
    myModel.addConstr(con2 <= 1) 
    if flag:
        print(quicksum(h[j,n,c] for j in range(1,n+1)))

$$
\sum_{j\neq0\prime}^{N}X_{jic}-\sum_{j\neq0}^{N}X_{ijc}=0,\ \forall i=1,...,N,\ \forall c
$$

In [13]:
for i in range(2,n):
    for c in range(1,nc+1):
        sum1 = quicksum(x[j,i,c] for j in range(1,n))
        sum2 = quicksum(x[i,j,c] for j in range(2,n+1))
        myModel.addConstr(sum1 - sum2 == 0) 
        if flag:
            print(i,c,quicksum(h[j,i,c] for j in range(1,n))-quicksum(h[i,j,c] for j in range(2,n+1)))

$$
\sum_{j\neq0}{\sum_{c} X_{ijc}=1,}\forall i=1,...,N
$$

In [14]:
for i in range(2,n):
    sum3 = quicksum(quicksum(x[i,j,c] for c in range(1,nc+1)) for j in range(2,n+1))
    myModel.addConstr(sum3 == 1)
    if flag:
        print(quicksum(quicksum(h[i,j,c] for c in range(1,nc+1)) for j in range(2,n+1)))

$$
\sum_{i\neq0\prime}{\sum_{c} X_{ijc}=1,}\forall j=1,...N
$$

In [15]:
for j in range(2,n):
    sum4 = quicksum(quicksum(x[i,j,c] for c in range(1,nc+1)) for i in range(1,n))
    myModel.addConstr(sum4 == 1)
    if flag:
        print(quicksum(quicksum(h[i,j,c] for c in range(1,nc+1)) for i in range(1,n)))

$$
\sum_{i}\sum_{j}{X_{ijc}\times\frac{d_{ij}}{v}\le12,\ \forall c}
$$

In [16]:
for c in range(1,nc+1):
    con3 = quicksum(quicksum(x[i,j,c]*d[i,j]/v for j in range(1,n+1)) for i in range(1,n+1))
    myModel.addConstr(con3 <= 12) 
    if flag:
        print(quicksum(quicksum(h[i,j,c]*d[i,j]/v for j in range(1,n+1)) for i in range(1,n+1)))

$$
a_i\sum_{j} X_{ijc}\le w_{ic}\le b_i\sum_{j}{X_{ijc},\ \forall i}=1,...N,\forall c
$$

In [17]:
for c in range(1,nc+1):
    for i in range(2,n):
        equ1 = a[i-1]*quicksum(x[i,j,c] for j in range(1,n+1))
        equ2 = b[i-1]*quicksum(x[i,j,c] for j in range(1,n+1))
        myModel.addConstr(w[i,c] >= equ1) 
        myModel.addConstr(w[i,c] <= equ2) 
        if flag:
            print(w2[i,c]-a[i-1]*quicksum(h[i,j,c] for j in range(1,n+1)),b[i-1]*quicksum(h[i,j,c] for j in range(1,n+1))-w2[i,c] ,i,c)

$$
w_{ic}+s_i+\frac{d_{ij}}{v}-w_{jc}\le B_{ij}(1-X_{ijc}),\ \forall i,j\in{0,0\prime,1,...N},\forall c
$$

In [18]:
for c in range(1,nc+1):
    for i in range(1,n+1):
        for j in range(1,n+1):
            myModel.addConstr(w[i,c] + d[i,j]/v - w[j,c] <= large*(1 - x[i,j,c]))
            if flag:
                print(w2[i,c] + d[i,j]/v - w2[j,c] <= large*(1 - h[i,j,c]))
#                 print(w[i,c] + d[i,j]/v - w[j,c])

$$
X_{ijc}\neq1\ if\ i=j,\forall c
$$

In [19]:
for c in range(1,nc+1):
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i == j:
                myModel.addConstr(x[i,j,c] == 0)
                if flag:
                    print(h[i,j,c])

$$
X_{0\prime j c}=0,\ \forall j,\forall c
$$

In [20]:
for c in range(1,nc+1):
    for j in range(1,n+1):
        myModel.addConstr(x[n,j,c] == 0)

$$
X_{i0c}=0\ \forall i,\forall c
$$

In [21]:
for c in range(1,nc+1):
    for i in range(1,n+1):
        myModel.addConstr(x[i,1,c] == 0)

## Objective

$$
min\ \sum_{j}\sum_{c} X_{0jc}
$$

In [22]:
obj = quicksum(quicksum(x[1,j,c]for c in range(1,nc+1)) for j in range(1,n+1))
myModel.setObjective(obj)

In [23]:
obj

<gurobi.LinExpr: x[1,1,1] + x[1,1,2] + x[1,1,3] + x[1,1,4] + x[1,1,5] + x[1,1,6] + x[1,1,7] + x[1,2,1] + x[1,2,2] + x[1,2,3] + x[1,2,4] + x[1,2,5] + x[1,2,6] + x[1,2,7] + x[1,3,1] + x[1,3,2] + x[1,3,3] + x[1,3,4] + x[1,3,5] + x[1,3,6] + x[1,3,7] + x[1,4,1] + x[1,4,2] + x[1,4,3] + x[1,4,4] + x[1,4,5] + x[1,4,6] + x[1,4,7] + x[1,5,1] + x[1,5,2] + x[1,5,3] + x[1,5,4] + x[1,5,5] + x[1,5,6] + x[1,5,7] + x[1,6,1] + x[1,6,2] + x[1,6,3] + x[1,6,4] + x[1,6,5] + x[1,6,6] + x[1,6,7] + x[1,7,1] + x[1,7,2] + x[1,7,3] + x[1,7,4] + x[1,7,5] + x[1,7,6] + x[1,7,7] + x[1,8,1] + x[1,8,2] + x[1,8,3] + x[1,8,4] + x[1,8,5] + x[1,8,6] + x[1,8,7] + x[1,9,1] + x[1,9,2] + x[1,9,3] + x[1,9,4] + x[1,9,5] + x[1,9,6] + x[1,9,7] + x[1,10,1] + x[1,10,2] + x[1,10,3] + x[1,10,4] + x[1,10,5] + x[1,10,6] + x[1,10,7] + x[1,11,1] + x[1,11,2] + x[1,11,3] + x[1,11,4] + x[1,11,5] + x[1,11,6] + x[1,11,7]>

## Optimization

In [24]:
myModel.optimize()

Optimize a model with 1320 rows, 924 columns and 6755 nonzeros
Variable types: 77 continuous, 847 integer (847 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 609 rows and 273 columns
Presolve time: 0.16s
Presolved: 711 rows, 651 columns, 9853 nonzeros
Variable types: 63 continuous, 588 integer (588 binary)

Root relaxation: objective 0.000000e+00, 135 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   19          -    0.00000      -     -    0s
H    0     0                       3.0000000    0.00000   100%     -    0s
     0     0    0.00000    0   20    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0   28    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0   

In [25]:
myModel.ObjBound

2.0

In [26]:
for key,i in x.items():
    if i.x == 1:
        print(i)

<gurobi.Var x[1,7,2] (value 1.0)>
<gurobi.Var x[1,9,5] (value 1.0)>
<gurobi.Var x[2,5,5] (value 1.0)>
<gurobi.Var x[3,6,5] (value 1.0)>
<gurobi.Var x[4,8,2] (value 1.0)>
<gurobi.Var x[5,3,5] (value 1.0)>
<gurobi.Var x[6,11,5] (value 1.0)>
<gurobi.Var x[7,10,2] (value 1.0)>
<gurobi.Var x[8,11,2] (value 1.0)>
<gurobi.Var x[9,2,5] (value 1.0)>
<gurobi.Var x[10,4,2] (value 1.0)>


In [27]:
for key,i in w.items():
    if i.x != 0:
        print(i)

<gurobi.Var w[2,5] (value 1.6499999999999773)>
<gurobi.Var w[3,5] (value 6.75)>
<gurobi.Var w[4,2] (value 5.649999999999977)>
<gurobi.Var w[5,5] (value 5.199999999999932)>
<gurobi.Var w[6,5] (value 10.5)>
<gurobi.Var w[7,2] (value 2.2500000000000018)>
<gurobi.Var w[8,2] (value 8.75)>
<gurobi.Var w[9,5] (value 0.5499999999999545)>
<gurobi.Var w[10,2] (value 3.5499999999999563)>
<gurobi.Var w[11,1] (value 12.0)>
<gurobi.Var w[11,2] (value 12.0)>
<gurobi.Var w[11,3] (value 12.0)>
<gurobi.Var w[11,4] (value 12.0)>
<gurobi.Var w[11,5] (value 12.0)>
<gurobi.Var w[11,6] (value 12.0)>
<gurobi.Var w[11,7] (value 12.0)>
